<a href="https://colab.research.google.com/github/Aromal-801/mrar07/blob/main/Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


In [10]:
data = pd.read_excel("/content/partpdf_1758763736656_airbnb.xlsx", sheet_name=0)
data.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,500,2008-06-26,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
1,500,2008-06-26,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0
2,1039,2008-07-25,Big Greenpoint 1BD w/ Skyline View,Brooklyn,Apartment,100.0,Entire home/apt,11222.0,1.0,1,4,149,100.0
3,1783,2008-08-12,Amazing Also,Manhattan,Apartment,100.0,Entire home/apt,10004.0,1.0,1,9,250,100.0
4,2078,2008-08-15,"Colorful, quiet, & near the subway!",Brooklyn,Apartment,90.0,Private room,11201.0,1.0,1,80,90,94.0


loading dataset

In [11]:
# Assuming your DataFrame is named df and the column is 'Host Since'
data['Host Since'] = pd.to_datetime(data['Host Since'], errors='coerce')
data.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,500,2008-06-26,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
1,500,2008-06-26,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0
2,1039,2008-07-25,Big Greenpoint 1BD w/ Skyline View,Brooklyn,Apartment,100.0,Entire home/apt,11222.0,1.0,1,4,149,100.0
3,1783,2008-08-12,Amazing Also,Manhattan,Apartment,100.0,Entire home/apt,10004.0,1.0,1,9,250,100.0
4,2078,2008-08-15,"Colorful, quiet, & near the subway!",Brooklyn,Apartment,90.0,Private room,11201.0,1.0,1,80,90,94.0


Handling Missing Data

In [12]:
# Identify review-related columns
review_cols = [col for col in data.columns if "Review" in col]

In [5]:
# Assuming data is your DataFrame
# Check missing values
missing_count = data['Review Scores Rating'].isnull().sum()
print(f"Missing values in Review Scores Rating: {missing_count}")

Missing values in Review Scores Rating: 8320


In [13]:
# Impute using median
if "Review Scores Rating" in data.columns:
    median_rating = data["Review Scores Rating"].median()
    data["Review Scores Rating"].fillna(median_rating, inplace=True)

/tmp/ipython-input-4289701211.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Review Scores Rating"].fillna(median_rating, inplace=True)


In [14]:
# Median imputation for other numeric review-related columns
for col in review_cols:
    if data[col].dtype in ['float64', 'int64']:  # numeric columns
        median_val = data[col].median()
        data[col].fillna(median_val, inplace=True)

/tmp/ipython-input-2165055151.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(median_val, inplace=True)


In [15]:
# Check if missing values are handled
print(data[review_cols].isnull().sum())

Review Scores Rating (bin)    0
Number Of Reviews             0
Review Scores Rating          0
dtype: int64


### Imputation Review Scores Rating

To address missing values in the review_scores_rating column  I used median imputation

#### Justification:
- **Robust to Outliers**: Review scores often exhibit skewed distributions due to extreme ratings (e.g., very low or very high). The median is less influenced by these outliers than the mean.
- **Preserves Central Tendency**: It reflects the typical rating more accurately when the data is not symmetrically distributed.
- **Data Distribution**: A histogram of the column showed a slight right skew, making the median a more appropriate measure of central tendency than the mean.

This approach ensures that the imputed values are representative of the majority of listings without being distorted by extreme values.

Feature Engineering

In [16]:
# Create the interaction feature
data["Neighbourhood_RoomType"] = data["Neighbourhood "] + "_" + data["Room Type"]
# Select features & target
target = "Price"
# Encode categorical variables using one-hot encoding
data_encoded_no_interaction = pd.get_dummies(
    data.drop(columns=["Neighbourhood_RoomType"]), drop_first=True
)
data_encoded_with_interaction = pd.get_dummies(data, drop_first=True)

# Ensure target column exists and no missing values
X_no_interaction = data_encoded_no_interaction.drop(columns=[target]).fillna(0)
y_no_interaction = data_encoded_no_interaction[target]

X_with_interaction = data_encoded_with_interaction.drop(columns=[target]).fillna(0)
y_with_interaction = data_encoded_with_interaction[target]


In [18]:
# Train-test split
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(
    X_no_interaction, y_no_interaction, test_size=0.2, random_state=42
)


In [3]:
#  Train Random Forest Regressor

rf_no = RandomForestRegressor(random_state=42)
rf_no.fit(X_train_no.drop(columns=['Host Since']), y_train_no)




NameError: name 'X_train_no' is not defined

In [4]:

#  Evaluate RMSE
y_pred_no = rf_no.predict(X_test_no)
y_pred_with = rf_with.predict(X_test_with)

rmse_no = np.sqrt(mean_squared_error(y_test_no, y_pred_no))
rmse_with = np.sqrt(mean_squared_error(y_test_with, y_pred_with))

print("RMSE without interaction feature:", rmse_no)
print("RMSE with interaction feature:", rmse_with)

NameError: name 'X_test_no' is not defined

Final Model Training and Evaluation

In [7]:


# Assuming df is your DataFrame and 'price' is the target
# Create the interaction feature
data['neighbourhood_roomtype'] = data['neighbourhood'] + "_" + data['room_type']

# One-hot encode the interaction feature
X = pd.get_dummies(data[['neighbourhood_roomtype', 'review_scores_rating', 'minimum_nights', 'availability_365']], drop_first=True)
y = data['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the final Random Forest Regressor
rf_final = RandomForestRegressor(random_state=42)
rf_final.fit(X_train, y_train)

# Predict and calculate RMSE
y_pred = rf_final.predict(X_test)
rmse_final = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Final RMSE: {rmse_final:.2f}")

NameError: name 'data' is not defined